# Step 1: Import required modules

In [2]:
#disable auto save, this sometimes hangs the browser
%autosave 0
import pandas as pd
import time
from pandas.tools.plotting import scatter_matrix
import numpy
import sklearn
import datetime
from sklearn import preprocessing
from sklearn_pandas import DataFrameMapper
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import preprocessing
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.neural_network import MLPClassifier
import operator 

# to supress printing of exponential notation in pandas
pd.options.display.float_format = '{:20,.2f}'.format

# avoid data truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Autosave disabled


/Users/haris/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Helper Functions

### Function to return true if array contains binary (zero and one) values only

In [3]:
def is_only_zero_and_one(array):
    return len(array) == 2 and ((array[0] == 0 and array[1] == 1) or ((array[0] == 1 and array[1] == 0)))

### Function to convert categorical features into binary
#### use this in future instead: http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html
#### need to do this because: https://stackoverflow.com/questions/24715230/can-sklearn-random-forest-directly-handle-categorical-features

In [4]:
# does not modify the original source, removes the original categorical columns
def convert_categorical_to_binary(data, categorical_columns):
    
    temp_data = data.copy()

    label_binarizer = []
    for col in categorical_columns:
        label_binarizer.append((col, sklearn.preprocessing.LabelBinarizer()))
        
    # df_out=True: output a data frame
    mapper_df = DataFrameMapper(label_binarizer, df_out=True)    
    # temp contains the new columns
    temp = mapper_df.fit_transform(temp_data)
    
    # print temp[temp.isnull().any(axis=1)]
    
    for col in temp.columns:
        temp_data[col] = numpy.array(temp[col])
    
    total_column_count = len(data.columns)
    for col in categorical_columns:
        total_column_count += len(data[col].unique())
        
    print 'new column count should be ' + str(len(temp_data.columns)) + ' and is ' + str(total_column_count)
    for col in categorical_columns:
        temp_data = temp_data.drop() += len(data[col].unique())
    return temp_data

SyntaxError: invalid syntax (<ipython-input-4-f74cf64c0984>, line 26)

### Function to rescale all non-binary features between 0 and 1

In [ ]:
# does not modify the original source
# categorical_columns are skipped
# if a column only has binary (0/1) values, it is skipped too
def rescale_non_binary_columns(data, categorical_columns):
    
    temp_data = data.copy()
    scaler = preprocessing.MinMaxScaler()
    for col in data.columns:
        if col not in categorical_columns and not is_only_zero_and_one(data[col].unique()):
            # print 'scaling ' + col
            temp_data[col] = scaler.fit_transform(temp_data[[col]])
            
    return temp_data

### Function to print confusion matrix

In [ ]:
def print_confusion_matrix(confusion_matrix, labels):
    records = len(labels)
    for row in range(records):
        print "-------------" + labels[row] + "-------------"
        total = 0
        for column in range(records):
            total += confusion_matrix[row][column]
        print 'total: ' + str(total)
        print 'correct: ' + str(confusion_matrix[row][row])
        for column in range(records):
            if confusion_matrix[row][column] != 0 and row != column:
                print labels[column] + ': ' + str(confusion_matrix[row][column])

### Function to print F scores

In [5]:
def print_f_scores(actual_labels, predictions, unique_labels):
    #http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score
    # Calculate metrics globally by counting the total true positives, false negatives and false positives.
    micro = metrics.f1_score(actual_labels, predictions, 
                                           labels=unique_labels, average='micro')
    print 'micro: ' + str(micro)
    # Calculate metrics for each label, and find their unweighted mean. This does not take label imbalance into account.
    print 'macro: '+ str(metrics.f1_score(actual_labels, predictions, 
                                          labels=unique_labels, average='macro'))
    # Calculate metrics for each label, and find their average, weighted by support (the number of true instances 
    # for each label). This alters ‘macro’ to account for label imbalance; it can result in an F-score that
    # is not between precision and recall.
    print 'weighted: ' + str(metrics.f1_score(actual_labels, predictions, 
                                              labels=unique_labels, average='weighted'))
    
    return micro

### Function to convert millis to hour/day/month/year

In [6]:
def seconds_to_info(seconds):
  dt = datetime.datetime.fromtimestamp(seconds)  

  return {
    "hour_of_day": dt.hour,
    "day_of_week": dt.weekday(),
    "day_of_month": dt.day,
    "month_of_year": dt.month
  }

### Function to convert categorical features to binary

In [7]:
# does not modify the original source
def convert_categorical_to_binary(data, categorical_columns):
    
    temp_data = data.copy()

    label_binarizer = []
    for col in categorical_columns:
        label_binarizer.append((col, sklearn.preprocessing.LabelBinarizer()))
        
    # df_out=True: output a data frame
    mapper_df = DataFrameMapper(label_binarizer, df_out=True)    
    # temp contains the new columns
    temp = mapper_df.fit_transform(temp_data)
    
    # print temp[temp.isnull().any(axis=1)]
    
    for col in temp.columns:
        temp_data[col] = numpy.array(temp[col])
    
    total_column_count = len(data.columns)
    for col in categorical_columns:
        total_column_count += len(data[col].unique())
        
    print 'new column count should be ' + str(len(temp_data.columns)) + ' and is ' + str(total_column_count)
    for col in categorical_columns:
        temp_data = temp_data.drop(col, 1)
    return temp_data

## Load the dataset

In [8]:
data = pd.read_csv("/Users/haris/Desktop/hacareem-khi-master 2/problem3/large.csv")
print "csv loaded"

csv loaded


### Remove duplicates

In [9]:
print 'rows and columns: ' + str(data.shape)
# remove duplicate rows
data = data.drop_duplicates()
print 'rows and columns after removing duplicates:' + str(data.shape)
print 'printing rows with null values'
print len(data[data.isnull().any(axis=1)])

rows and columns: (4768, 11)
rows and columns after removing duplicates:(4768, 11)
printing rows with null values
0


### Unique users

In [10]:
print len(data['user_id'].unique())

174


### Remove rides where pickup and dropoff location is same

In [11]:
data = data.loc[data['pick_up_geohash'] != data['drop_off_geohash']]
len(data)

4748

### Extract hour/day/week.. info from the timestamp

In [12]:
hour_of_day = []
day_of_week = []
day_of_month = []
month_of_year = []

for index in range(0, len(data)):
    date_info = seconds_to_info(data.iloc[index]['pick_up_time'])

    hour_of_day.append(date_info['hour_of_day'])
    day_of_week.append(date_info['day_of_week'])
    day_of_month.append(date_info['day_of_month'])
    month_of_year.append(date_info['month_of_year'])
    
data['hour_of_day'] = hour_of_day
data['day_of_week'] = day_of_week
data['day_of_month'] = day_of_month
data['month_of_year'] = month_of_year

### Find uniuqe pick_up vs unique pick_up_geohash

In [13]:
print len(data['pick_up'].unique())
#print len(data['pick_up_lng'].unique())
print len(data['pick_up_geohash'].unique())

972
2489


### Removing unnecessary columns 

In [14]:
columns_to_remove = [
    'ride_id',  
    'pick_up_time',
    'pick_up_lat',
    'pick_up_lng',
    'pick_up_geohash',
    'drop_off_lat',
    'drop_off_lng',
    'drop_off_geohash',
    'day_of_month',
    'month_of_year'
]

for col in columns_to_remove:
    data = data.drop(col, 1)

In [15]:
#remaining columms
data.columns

Index([u'user_id', u'pick_up', u'drop_off', u'hour_of_day', u'day_of_week'], dtype='object')

### Train classifier for each user

In [16]:
label_column = 'drop_off'
random_state=42

micro = 0.0
total = 0
# train a separate classifier for each user
for user in data['user_id'].unique():
    print '-------------- NEW USER-----------------------'
    local_data = data.copy()
    labels = local_data[label_column].unique()
    user_data = local_data[local_data['user_id'] == user]
    user_data = user_data.drop('user_id', 1)

    if len(user_data) < 50:
        continue
        
    print 'user: ' + user
    print 'examples: ' + str(len(user_data))
        
    for label in labels:
        if len(user_data[user_data['drop_off'] == label]) < 2:
            user_data = user_data[user_data['drop_off'] != label]
    
    print 'examples after removing y with count < 2: ' + str(len(user_data))
    if len(user_data) == 0:
        continue;

    user_data = convert_categorical_to_binary(user_data, ['pick_up'])            
    labels = user_data[label_column].unique()
    y = user_data[label_column]
    X = user_data.drop(label_column, 1)
        
    if (len(X) * 0.20) < len(labels):
        continue;
    
    X_train, X_cross_validation, y_train, y_cross_validation = train_test_split (X, y, test_size=0.2, 
                                                                                 random_state=random_state, stratify = y)
            
    random_forest_classifier = RandomForestClassifier(n_estimators=50, random_state=random_state)
    random_forest_classifier = random_forest_classifier.fit(X_train, y_train)
    random_forest_predictions = random_forest_classifier.predict(X_cross_validation)
    
    confusion_matrix = metrics.confusion_matrix(y_cross_validation, random_forest_predictions, labels=labels)
    #print_confusion_matrix(confusion_matrix, labels)
    micro = micro + print_f_scores(y_cross_validation, random_forest_predictions, labels)
    total = total + 1
    
    #if user == '2d1bafe55b':
    #    print X_train
    #    break
        
print 'overall accuracy: ' + str(micro/total)

-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
user: c8c41c4a18
examples: 263
examples after removing y with count < 2: 245
new column count should be 36 and is 36
micro: 0.836734693878
macro: 0.267165314402
weighted: 0.783146361993
-------------- NEW USER-----------------------
user: cc6ef8cb8d
examples: 158


/Users/haris/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/haris/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


examples after removing y with count < 2: 123
new column count should be 41 and is 41
micro: 0.36
macro: 0.117647058824
weighted: 0.344705882353
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
user: daec4b0d54
examples: 82
examples after removing y with count < 2: 68
new column count should be 18 and is 18
micro: 0.642857142857
macro: 0.307692307692
weighted: 0.571428571429
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
-------------- NEW USER-----------------------
user: 51d967ebc7
examples: 65
examples after removing y with count < 2: 51
new column count should be 17 and is 17
micro: 0.636363636364
macro: 0.341991341991
weighted: 0.603305785124
-------------- NEW USER-----------------------
-------------- NE